This is a how to guide for importing data from .csv file to Django database (SQLite). You will see example data classes. Please note that those classes are for demonstration only. You are free to create classes in the way you see fit the best. 

The end goal is to create a JSON data that will be used to import data by Django built-in command. The format we want is structured as below. (more detail check -> https://docs.djangoproject.com/en/5.1/howto/initial-data/)

```JSON
[
  {
    "model": "myapp.classname",
    "pk": 1,
    "fields": {
      "attribute_name1": "Value",
      "attribute_name2": "Value"
    }
  },
  {
    "model": "myapp.field",
    "pk": 2,
    "fields": {
      "field_id": "32",
      "field_name": "Corner"
    }
  }
]

```

In Python, you can assume that JSON is Dictionary-like data type. So, the overall step is .csv -> Pandas dataframe -> dictionary ->JSON. 

Let's start with importing necessary modules. In this case, we only need 2 modules. Then load data from .csv to Pandas dataframe (we did this before in lab 2).

In [ ]:
# import python's module
import pandas as pd
import json

In [ ]:
# load data
log_df = pd.read_csv("operation-log.csv")  
log_df
# _df stands for data frame. It is a common suffix to indicate variable type. 

First data class I want to create is Operator class ass below. As you can see that I created a class with attributes that we don't have data. That is okay, we can leave those attributes optional by adding `blank=True, null=True` (more detail here -> https://docs.djangoproject.com/en/5.1/ref/models/fields/#field-options).

```python 
class Operator(models.Model):
    firstname = models.CharField(max_length=30)
    lastname = models.CharField(max_length=30, blank=True, null=True)
    middlename = models.CharField(max_length=30, blank=True, null=True)
    phone = models.CharField(max_length=10, blank=True, null=True)
```

You can think of a database class as a table. So, let's create an `Operator` table (dataframe). We will use Pandas' Unique fucntion to see how many operator rows we need to create. (more detail about unique function -> https://pandas.pydata.org/docs/reference/api/pandas.unique.html)

In [ ]:
log_df['operator'].unique() # find a unique value in dataframe's `operator` column

The result from unique function is an array which is iterable. We will iterate over the array and create a dataframe. There are several ways to create a dataframe. But I recommend doing by using a list of dict or a dict of list. I will show you the first case. 

In [ ]:
# create a list which we will add dictionaries later
operators = [] # suffix s is common way to indicate a list (array)
for person in log_df['operator'].unique(): # we iterate over the result from unique function
    operators.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "firstname": person, # each key-value pair
            "lastname": "",
            "phone": ""
        } # the end of dictionary
    )
operators # check the outcome

Now we have a list of dictionaries. Making a dataframe is just one function. Note that I leave `lastname` and `phone` emtyp. This is an explicit way to work with optional attribute. You can also just ignore that attribute like I did with `middlename`. You will get the same outcome. 

In [ ]:
# create a dataframe
operator_df = pd.DataFrame(operators)
operator_df

The next step is to structure the data as required (check data format above). We will need to add `model` which is the class's name in the database. And `pk` or primary key which is the row's index. We want JSON at the end, but JSON is pretty much like dictionary in Python. So, we just need to create a (list of) dictionary.

In [ ]:
operators = [] # create an empty list
for index, row in operator_df.iterrows(): # we iterate over the dataframe row by row
    operators.append( # append new element to the list
        { # curly bracket indicate the starting of dictionary
            "model": "acrelog.operator",  # the class name
            "pk": index+1, # index starts at 0 while primary key starts at 1. 
            "fields": { # this is a dictionary of attributes
                "firstname": row['firstname'] # only need firstname for now
            }
        } # the end of dictionary
    )
operators # see the result

Now we have the data in the format that we need. To convert into JSON, you can just run the following command. Then you can copy the result and check with https://jsonlint.com/

In [ ]:
json.dumps(operators)

In [ ]:
# or write the JSON to the file directly 
with open('operator-data.json', 'w') as fp:
    json.dump(operators, fp=fp, indent=4)

You might wonder why we create a dictionary then convert into a dataframe just for turn it into a dictionary again. Can you just make a dictionary that follows the required stucture directly? Absolutely yes. In some cases, that will be the better way to deal with the data. But there is a usecase for a dataframe as well. You will see it soon. For now, let's create another data class. 

```python
class Operation(models.Model):
    date = models.DateField()
    note = models.CharField(max_length=300, blank=True, null=True)
    operator = models.ForeignKey(Operator, on_delete=models.CASCADE)
```
In this class, it has a foreignkey that points to Operator class. So, instead of recording operator name, we will keep the primary key that points to that operator. For better understanding, I will trim the original dataframe to keep only the data that we will play with.

In [ ]:
# you can subset the dataframe with any columns (and order)
sub_log_df = log_df[['date', 'operator', 'note']]
sub_log_df

We will use the map function (more detail -> https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html) to map from the operator's name to primary key. The map functions take several options. But we will use dictionary. The dictionary we want has operator's names as keys and primary keys as values. Do you feel that this structure looks familar. Yes, that is the operator dataframe that we created earlier. 

In [ ]:
operator_df

As I mentioned earlier, dataframe is made from a list of dictionaries or a dictionary of lists. We can convert it back as well.

In [ ]:
operator_df.to_dict()

The format you see above is a dictionary of list. We want to use firstname to map with primary key (or dataframe's index). So we will only care about the `firstname`.

In [ ]:
operator_df.to_dict()['firstname']

Almost done. We want a dictionary that keys are firstname and values are primary keys. But what we got is opposite. No problem, we just need to inverse it (and add 1 to pandas' index to make it become primary key)

In [ ]:
index_name = operator_df.to_dict()['firstname']
name2pk = dict((v, k+1) for k, v in index_name.items())
name2pk

We got it. Now, let map the operator's name to primary key.

In [ ]:
sub_log_df['operator'] = sub_log_df['operator'].map(name2pk) 
sub_log_df

Before we make the final dictionary, we need to deal with date format. Django's SQLite expects `YYYY-MM-DD` format. So, we need to fix our date column a little bit. What you see is only string (a series of charactor). Python has a date (or datetime) variable for dealing with this type of data. Date data type is numerial. So, we can do calulation and also, in our case, format in the way we want. First, we need to convert string to datetime. 

In [ ]:
pd.to_datetime(sub_log_df['date'])

As you can see, we have a date column which we can do many thing on it. But we will just convert it back into string (with the format that we want). Then we will make a dictionary and JSON. The function we use is `strftime` (string from time).

In [ ]:
sub_log_df['date'] = pd.to_datetime(sub_log_df['date']).dt.strftime('%Y-%m-%d')
sub_log_df

Now, we have everything ready. Let's create the final dictionary.

In [ ]:
logs = []
for index, row in sub_log_df.iterrows():
    logs.append({
        "model": "acrelog.operation", 
        "pk": index+1,
        "fields": {
            "date": row['date'],
            "operator": row['operator'],
            "note": row['note'],
        }
    })
logs

You can dump the dictionary to JSON then load the JSON file to the database.